In [1]:

from functools import partial
import nltk

import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
seed_everything(1376)

Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [9]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [10]:
torch.ones_like(z).device

device(type='cuda', index=0)

In [2]:
configs = {}
configs["model_base"] = "facebook/bart-base"
configs["task_type"] = "xsum"

In [3]:
tokenizer = create_text_tokenizer(configs["model_base"])

task_dataset_gen = get_dataset_specified_tasks(configs["task_type"])

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs["model_base"],
        is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
generator = model_builder()
train_model_path = "trained_models_sum_boundary/bart_base_model_baseline/checkpoint-25506/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")#


In [5]:
te1= test_dataset[6]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

Token indices sequence length is longer than the specified maximum sequence length for this model (1187 > 1024). Running this sequence through the model will result in indexing errors


In [6]:
seed_everything(1376)

Global seed set to 1376


1376

In [7]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=129,
        early_stopping=True,
        use_cache=True,
        num_beams=5,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['Wendy Houvenaghel says British Cycling "has acknowledged and takes very seriously previous cultural and governance failings" in its World Class programme.']

In [7]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'A "medal at any cost" approach created a "culture of fear" at British Cycling, says former rider Wendy Houvenaghel.'

In [51]:
te1.input_ids.shape

torch.Size([1187])

In [58]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

Media playback is not supported on this device
The Olympic silver medallist accused the organisation of "ageism" and having "zero regard" for her welfare.
She is the latest high-profile cyclist to come forward after Jess Varnish, Nicole Cooke and Emma Pooley criticised the World Class programme.
Houvenaghel told the BBC she felt "vindicated" by a leaked draft report detailing British Cycling's failures.
The report said British Cycling "sanitised" its own investigation into claims former technical director Shane Sutton used sexist language towards Varnish, who went public last April about her treatment.
British Cycling subsequently admitted it did not pay "sufficient care and attention" to the wellbeing of staff and athletes at the expense of winning medals, an approach Houvenaghel attested to in her BBC interview.
Both Sutton and predecessor Sir Dave Brailsford have now left British Cycling.
Houvenaghel, 42, spoke to BBC Sport during its State of Sport week, which on Thursday examines 

In [31]:
boundary_mask

(360, 610)

In [8]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
test_data_loader = DataLoader(test_dataset,batch_size=12,
                              sampler= SequentialSampler(test_dataset),
                              collate_fn= SmartCollator(
            pad_token_id=train_dataset.tokenizer.pad_token_id, max_len=800,
            context_max_len=configs.c,
            context_sampling_bounds=(0.1, 0.45)
        )
                              )

output_summaries2 =[]
for batch in tqdm.tqdm(test_data_loader):
    b_input_ids = batch['input_ids'].to(device)
    b_input_mask = batch['attention_mask'].to(device)
    
    seq_len = b_input_ids.shape[1]
    
    boundary_mask =  batch.get("boundary",)
    bb=generator.generate(input_ids=b_input_ids,
            attention_mask=b_input_mask,
            num_beams=10,
            do_sample=False,
            num_return_sequences=1,
            max_length=140)
    sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
    output_summaries2+=sentences

100%|██████████| 945/945 [10:56<00:00,  1.44it/s]


In [31]:
boundary_mask

(360, 470)

In [9]:
#context_enforcement/data/common.py
from context_enforcement.data.common import write_to_file
write_to_file(output_summaries2[:len(test_dataset)], "outputs/attempt1/baseline_2")

Done


In [10]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

In [11]:
targets[690],output_summaries2[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 'Kenyan authorities have arrested Congolese singer Emmanuel Olomide in the capital, Kinshasa, on suspicion of assault, the BBC understands.')

In [12]:
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
scores4 = metrics.compute(predictions=output_summaries2,references=targets)

In [13]:
scores = metrics.compute(predictions=output_summaries2,references=targets)

In [14]:
scores

{'bleu': 0.13001522827171397,
 'precisions': [0.43264780948765,
  0.17764752271710285,
  0.09726789720912225,
  0.057229256856921935],
 'brevity_penalty': 0.90401138133862,
 'length_ratio': 0.9083367181931746,
 'translation_length': 237045,
 'reference_length': 260966,
 'meteor': 0.3623311725177827,
 'rouge1': 0.40646161409356885,
 'rouge2': 0.18746257090958113,
 'rougeL': 0.3322105156464427,
 'rougeLsum': 0.33234181119671774}

In [30]:
scores

{'bleu': 0.08190144303338862,
 'precisions': [0.39916558741021463,
  0.13484576712224944,
  0.06490478666102122,
  0.03461400195153127],
 'brevity_penalty': 0.7810193894484071,
 'length_ratio': 0.8018247587808374,
 'translation_length': 209249,
 'reference_length': 260966,
 'meteor': 0.28558238733460617,
 'rouge1': 0.34475899874216664,
 'rouge2': 0.13444853158452552,
 'rougeL': 0.279360220097055,
 'rougeLsum': 0.27932587220115546}

In [39]:
scores2

{'bleu': 0.08004077243766287,
 'precisions': [0.39668147524032016,
  0.1326978952590178,
  0.06327462141553002,
  0.03340461786535229],
 'brevity_penalty': 0.7793378458842136,
 'length_ratio': 0.8004414368155238,
 'translation_length': 208888,
 'reference_length': 260966,
 'meteor': 0.28278544337874784,
 'rouge1': 0.34230456443341767,
 'rouge2': 0.13258619227287083,
 'rougeL': 0.27739509890876,
 'rougeLsum': 0.2774115466752346}

In [28]:
scores3

{'bleu': 0.11312982003210258,
 'precisions': [0.43367935409457903,
  0.170965703452634,
  0.09041255378385218,
  0.051220088499377064],
 'brevity_penalty': 0.8310753377221631,
 'length_ratio': 0.8438570541756397,
 'translation_length': 220218,
 'reference_length': 260966,
 'meteor': 0.3379248069191188,
 'rouge1': 0.3902876664001791,
 'rouge2': 0.17375564911724187,
 'rougeL': 0.3204999917403623,
 'rougeLsum': 0.3205687202163906}

In [35]:
scores4

{'bleu': 0.11146082917260128,
 'precisions': [0.4317755562839142,
  0.1685194251648763,
  0.08904895332074475,
  0.050525125222168364],
 'brevity_penalty': 0.8286310630214466,
 'length_ratio': 0.8417648276020631,
 'translation_length': 219672,
 'reference_length': 260966,
 'meteor': 0.334512237524537,
 'rouge1': 0.3879467178046803,
 'rouge2': 0.17119907794549927,
 'rougeL': 0.31801217096792334,
 'rougeLsum': 0.31809606706132665}